In [2]:
import pandas as pd
def make_ques_doc(experiment):
    human_data_file = pd.read_csv(experiment + "_counts.csv")
    
    with open("words_" + experiment + ".txt", 'w') as new_file:
        new_file.truncate(0)
        for i in range(len(human_data_file["prompt"])):
            new_file.write(human_data_file["prompt_w1"][i] + " ")
            new_file.write(human_data_file["prompt_w2"][i] + " ")
            new_file.write(human_data_file["prompt_w3"][i] + " ")

            for r in human_data_file.columns[4:]:
                if human_data_file[r][i] > 0:
                    new_file.write(r + " ")
            new_file.write("\n")

        new_file.close()

make_ques_doc("experiment1a")
make_ques_doc("experiment1b")
make_ques_doc("experiment1c")

In [3]:
#read in csv file
#open text file with PG val from first column of csv file

import os
import shutil as sh

def create_text_folder(genre):
    corpus = pd.read_csv(genre + "_texts.csv")
    texts = corpus["id"]
    
    #path will be changed
    path = genre + "_docs"
    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    if not isExist:
        os.makedirs(path)
    for i in range(len(texts)):
        sh.copy("/Users/erikajones/Documents/CS 8803/LSA Project/text_database/gutenberg/data/text/" + texts[i] + "_text.txt", path)

create_text_folder("nonfiction")
create_text_folder("fiction")
create_text_folder("poetry")
        

In [6]:
import glob
def find_analogy_question(experiment, genre):
        
    with open(experiment + "_" + genre + "_analogy_questions.txt", "w") as analogies:
        analogies.truncate(0)
        f = open("words_" + experiment + ".txt", 'r')
        questions = f.readlines()
        questions_info = []
        words_in_doc = True
        
        for line in questions:
            line = line.split()
            line = line[0:8]
            words_in_docs = True
            count_sum = 0
            word_info = []
            
            for word in line:
                word_count = 0
                
                for text_file in glob.glob(genre + "_docs" + "/*"):
                    if word in open(text_file).read():
                        word_count += 1
                if word_count == 0:
                    words_in_docs = False
                    
                word_info.append((word_count,word))   
                count_sum += word_count
            word_info.sort()
            if words_in_docs:
                questions_info.append((count_sum, (line[0],line[1],line[2]), word_info))
                
        questions_info.sort()
            
        for q in questions_info:
            analogies.write(str(q) + "\n")

find_analogy_question("experiment1a", "nonfiction")
find_analogy_question("experiment1b", "nonfiction")
find_analogy_question("experiment1c", "nonfiction")
find_analogy_question("experiment1a", "fiction")
find_analogy_question("experiment1b", "fiction")
find_analogy_question("experiment1c", "fiction")
find_analogy_question("experiment1a", "poetry")
find_analogy_question("experiment1b", "poetry")
find_analogy_question("experiment1c", "poetry")

thermometer
temperature
barometer
pressure
humidity
wind
air
pressure
furnace
coal
stomach
food
acid
heat
ulcers
digestive
father
son
mother
daughter
father
sugar
coffee
honey
tea
milk
toast
taste
bees
knee
kneepad
snail
shell
snail
slow
salt
french
blizzard
snowflake
monsoon
tornado
raindrop
rain
water
mon
movie
screen
lightning
sky
storm
bolt
camera
thunder
pen
pig
coop
chicken
hen
pencil
chickens
baker
cake
scientist
experiment
science
theory
formula
lab
aspirin
pain
antacid
acid
heartburn
indigestion
ache
stomach
hoof
hoofprint
paw
pawprint
leg
cat
paw
claw
ketchup
tomato
guacamole
avocado
avacado
dip
guacamole
salad
bracelet
wrist
moat
water
castle
arm
moat
fort
revising
manuscript
evolving
life
species
growing
organism
change
ash
fireplace
soot
water
chimney
furnace
damp
candle
burger
bun
book
story
cover
shelf
pages
page
watermelon
rind
orange
peel
skin
apple
pulp
juicy
flock
goose
constellation
star
stars
concept
vulpecula
jacket
zipper
wound
skin
stitches
bandage
bandaid
blood

receptionist
office
company
store
desk
restaurant
cat
animal
canary
bird
animal
elf
santa
bodyguard
celebrity
president
client
boss
protect
swimming
sport
golf
sport
hobby
game
leisure
athlete
milk
carton
car
garage
automobile
gasoline
metal
shop
cow
barn
chicken
coop
coup
henhouse
coupe
farm
headphones
listen
ear
hear
listen
audio
piece
sound
team
player
squad
company
member
soldier
people
cheerleader
rifle
hunt
gun
protect
shoot
kill
hunt
target
concierge
hotel
doorman
lobby
apartment
hotel
building
hilton
shoe
leather
eraser
rubber
pencil
pink
plastic
tape
fix
gauze
protect
heal
wrap
mend
bandage
toy
object
John
person
name
man
doe
human
tongue
lick
teeth
bite
chew
shark
tank
cereal
box
bowl
cupboard
milk
broom
sweep
mop
mop
clean
wipe
mops
wash
school
educate
fan
game
cool
blow
blows
support
diamond
jewel
gold
metal
treasure
bar
element
material
cartridge
printer
pickle
jar
brine
seed
sandwich
burger
miner
quarry
clerk
office
store
desk
shop
bank
ostrich
bird
gold
metal
treasure
ba

mix
peeler
vegetable
peel
potato
orange
rind
blanket
cloth
basket
plastic
ball
container
cane
wood
sweater
wool
sock
cloth
thread
shoe
cotton
outfit
calculator
calculate
tongue
tongue
taste
lick
talk
speak
boat
harbor
bowl
cupboard
food
table
kitchen
shelf
innkeeper
inn
maid
hotel
motel
clean
house
home
dish
cabinet
bowl
cupboard
closet
sink
fruit
rack
salad
lettuce
city
people
building
town
street
buildings
flipper
swim
oar
boat
row
paddle
sail
rowing
gun
holster
book
cover
bag
library
shelf
case
snowman
snow
ice
water
ice
cream
rain
cap
flower
vase
plant
seed
pot
dirt
garden
vase
editor
magazine
journalist
story
newspaper
media
paper
news
nest
twig
flock
bird
animal
sheep
birds
geese
cookie
jar
cereal
box
bowl
milk
breakfast
cancer
illness
fedora
clothing
cap
hat
jacket
myeloma
(161, ('photographer', 'magazine', 'cartoonist'), [(1, 'cartoonist'), (5, 'cartoon'), (5, 'photographer'), (23, 'comic'), (23, 'comic'), (28, 'newspaper'), (38, 'magazine'), (38, 'magazine')])
(215, ('clown', 

mechanic
wrench
tools
tool
machines
drawer
laureate
dishonor
drought
flood
dry
wet
rain
downpour
enigma
puzzlement
fire
water
flame
hot
burn
heat
retirement
pension
puberty
growth
hormones
adult
acne
allowance
love
hate
fail
succeed
win
lose
pass
in
stream
river
sadness
depression
tears
sorrow
emotion
grief
poem
anthology
bird
flock
animal
fly
nest
nests
famine
plenitude
depression
happy
happiness
joy
mania
contentment
ship
fleet
soldier
army
platoon
battalion
squadron
squad
practice
improvement
germs
sickness
disease
illness
sick
soap
old
young
difficult
easy
easygoing
new
hard
rage
emotion
robin
happy
bird
batman
hood
hero
brave
water
wet
liquid
smooth
blue
cool
Elizabeth
queen
Ireland
country
king
drunkards
nation
leprochaun
conductor
baton
soldier
army
gun
rifle
war
salute
pardon
sin
row
boat
column
vertical
stadium
apology
engine
car
finger
hand
body
nail
happy
sad
short
tall
long
angry
minimi
Everest
mountain
Dallas
river
city
state
texas
cowboys
attack
defend
dirty
clean
wash
sp

credential
work
authority
rage
emotion
spear
tool
gun
weapon
fight
stick
car
auto
kid
person
child
children
human
plow
earth
brush
hair
tree
trim
canvas
teeth
retirement
pension
childhood
allowance
adulthood
school
fun
support
seminary
theologian
zoo
animal
home
zoologist
zookeeper
animals
infancy
cradle
adulthood
maturity
bed
sleep
work
home
enigma
puzzlement
disease
sickness
illness
sick
death
symptoms
sound
inaudible
empty
full
nothing
fulfill
fill
scarce
portrait
person
trophy
win
team
success
sport
champion
novice
inexperience
coward
scared
ignorant
fear
weakness
professional
lateness
hurry
dirtiness
dirty
clean
wash
shower
cleaning
portrait
person
summary
book
story
words
paper
article
beggar
poor
bigot
angry
ignorant
dumb
racist
hateful
scepter
authority
crown
king
allegiance
tyranny
power
authority
epilogue
novel
toe
body
nail
foot
leg
toenail
smooth
rough
dark
light
bright
black
Nile
river
Earth
land
planet
universe
happy
sad
fast
slow
speed
arsenal
weapon
garden
tool
trees
ve

cruel
enemy
backdrop
vista
summary
book
story
short
paragraph
overview
famine
plenitude
prisoner
sadness
freedom
free
freeman
civilian
bookshelf
books
aquarium
water
fish
fatigue
sleep
tiredness
lazy
slow
sleep
rest
nap
signature
approval
cross
reject
denial
symbol
sign
religion
beggar
poor
candy
rich
kid
sweet
sugar
cavities
portrait
person
biography
person
story
life
author
celebrity
sing
dirge
paint
landscape
color
picture
brush
canvas
joke
laughter
loss
fall
tears
grief
cry
upset
Nile
river
Yale
school
university
college
univerisiy
mountains
Everest
mountain
Atlantic
ocean
enigma
puzzlement
exercise
health
run
game
work
sweat
famine
plenitude
tyranny
peace
freedom
kindness
democracy
famine
sing
dirge
row
boat
column
fight
swim
paddle
conductor
baton
musician
music
singer
instrument
guitar
stick
buy
sell
criticize
approve
praise
enjoy
acclaim
help
enthusiasm
fervor
jog
run
stop
sweat
exercise
memory
simmer
boil
road
city
rage
drive
street
highway
practice
improvement
explosion
loud


sun
planet
world
round
happy
sad
dark
light
bright
scary
fair
arsenal
weapon
school
book
run
learn
children
education
harmony
discordant
enthusiasm
depression
melancholy
joy
excitement
sadness
laureate
dishonor
novice
new
beginner
learn
experience
expert
rapid
quick
teach
train
learn
educate
instruct
show
hunger
eat
dirtiness
clean
wash
shower
bathe
filthy
diary
person
recording
voice
music
singer
sound
musician
dupe
gullibility
coward
scared
cower
fear
weakness
brave
tree
forest
book
paper
bookshelf
bible
library
read
scepter
authority
cross
death
freedom
religion
holiness
church
sonnet
poem
red
blue
green
color
blood
rose
famine
plenitude
weakness
sorrow
strength
strong
healthy
beggar
poverty
idiot
stupidity
dumb
stupid
unintelligent
fool
buy
sell
go
stop
come
stay
far
return
beggar
poor
professor
rich
teacher
student
smart
intelligent
attack
defend
joy
hate
sorrow
happy
sadness
smile
farmer
tractor
shooter
gun
rifle
officer
gunman
target
seminary
theologian
refrigerator
cold
food
fo